In [1]:
import pandas as pd
import yaml

MY_SITE_YAML = "./test.yml"

In [2]:
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/project_prioritization/"
FILE = "fake_data.xlsx"

In [3]:
df = pd.read_excel(f"{GCS_FILE_PATH}{FILE}", sheet_name="fake")

In [4]:
districts = sorted(df.district.fillna(0).unique().tolist())
counties = sorted(df.full_county_name.fillna("Various").unique().tolist())

In [5]:
with open(MY_SITE_YAML) as analyses:
    analyses_data = yaml.load(analyses, yaml.Loader)

In [6]:
analyses_data

{'directory': './project_prioritization/',
 'notebook': './project_prioritization/my-notebook.ipynb',
 'parts': [{'chapters': [{'caption': 'District 1',
     'params': {'district': 1},
     'sections': [{'county': 'Los Angeles'}]},
    {'caption': 'District 2',
     'params': {'district': 2},
     'sections': [{'county': 'Los Angeles'}]}]}]}

In [7]:
test_districts = districts[:2]

In [8]:
# Loop through each district, grab the valid itp_ids
# populate each dict key (caption, params, sections) needed to go into analyses.yml
chapters_list = []
for district in sorted(test_districts):
    chapter_dict = {}
    
    subset = df[df.district == district]
    
    # Only grab the counties found in this district
    counties_in_district = sorted(subset.full_county_name.fillna("Various")
                                   .unique().tolist())
    
    chapter_dict['caption'] = f'District {district}'
    chapter_dict['params'] = {'district': district}
    chapter_dict['sections'] = [{'county': county} for county in 
                                counties_in_district]
    chapters_list += [chapter_dict]

# Make this into a list item
parts_list = [{'chapters': chapters_list}]

analyses_data['parts'] = parts_list

output = yaml.dump(analyses_data)

print(output)

directory: ./project_prioritization/
notebook: ./project_prioritization/my-notebook.ipynb
parts:
- chapters:
  - caption: District 0
    params:
      district: 0
    sections:
    - county: Various
  - caption: District 1
    params:
      district: 1
    sections:
    - county: Del Norte
    - county: Humboldt
    - county: Lake
    - county: Mendocino
    - county: Various



In [9]:
with open("./test2.yml", 'w') as analyses:
    analyses.write(output)